In [1]:
import cPickle
import numpy as np
import tensorflow as tf

print "loading data..."
x = cPickle.load(open("Data/imdb-train-val-test.pickle", "rb"))
revs, word2vec, word_idx_map = x[0], x[1], x[2]
print "data loaded!"

loading data...
data loaded!


In [2]:
word2vec.shape

(150855, 300)

In [3]:
import pandas as pd
def sentence2_1hot(sentence, max_len, word_idx_map):
    V=np.zeros(shape=max_len, dtype=np.int32)
    i=0
    for word in sentence.split():
        V[i]=word_idx_map[word]
        i+=1
        if i>=35:
            break
    return V
max_sentence_len=np.max((pd.DataFrame(revs)['num_words']))

In [4]:
tmp=sentence2_1hot("my name is", 35, word_idx_map)
tmp

array([32081, 17842, 28011,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0], dtype=int32)

In [5]:
len(revs)

200000

In [6]:
def embedding_batch(revs, W, max_sentence_len=35, batch_size=20, batch_index=1, embedding_size=300):
    tail=batch_size*(batch_index+1)
    assert tail<=len(revs)
    embedded=np.zeros(shape=(batch_size, max_sentence_len, embedding_size, 1), dtype=np.float32)
    labels=np.zeros(shape=batch_size, dtype=np.int32)
    k=0
    for i in range(tail-batch_size, tail):
        sentence_1hot=sentence2_1hot(revs[i]['text'], max_sentence_len, word_idx_map)
        for j in range(len(sentence_1hot)):
            embedded[k][j][:][:]=W[sentence_1hot[j]][:embedding_size].reshape(embedding_size,1)
        labels[k]=int(revs[i]['y']*0.25)
        k+=1
    return embedded, labels

In [7]:
embedding_size=50
X_train_num, y_train = embedding_batch(revs, word2vec, batch_size=1500, batch_index=0, embedding_size=embedding_size)
X_valid_num, y_valid = embedding_batch(revs, word2vec, batch_size=200, batch_index=20, embedding_size=embedding_size)

In [8]:
m,n=0,0

for i in y_train:
    if i==0:
        m+=1.0
    if i==1:
        n+=1.0
print(m/(m+n))

0.508666666667


In [9]:
m,n=0,0

for i in y_valid:
    if i==0:
        m+=1.0
    if i==1:
        n+=1.0
print(m/(m+n))

0.525


In [ ]:
import tensorflow as tf
import time

X=tf.placeholder(tf.float32, shape=[None, 35, embedding_size,1], name="X_input")
y=tf.placeholder(tf.int64, shape=None, name="y_input")

In [ ]:
# #Embedding layer, embedding size set to 300
# embedding_size=100
# with tf.name_scope("embedding_layer"):
#     W=tf.Variable(tf.random_uniform([len(vocabulary), embedding_size], -1.0,1.0), name="W")
#     #W=tf.Variable(tf.random_uniform([5000, embedding_size], -1.0,1.0), name="W")
#     embedded_chars=tf.nn.embedding_lookup(W, X)
#     embedded_chars_expanded=tf.expand_dims(embedded_chars, -1)


#embedded_chars_expanded=tf.expand_dims(X,-1)
#Convolution layer
#embedded_chars_expanded=X

# y_train=np.array(y_train, dtype=np.int32)
# y_valid=np.array(y_valid, dtype=np.int32)
pooled_outputs=[]
n_filters=10

with tf.name_scope("conv_layer"):
    filter_shape=[3, embedding_size, 1, n_filters]
    W=tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="Weights")
    b=tf.Variable(tf.constant(0.1, shape=[n_filters]), name="bias")
    conv=tf.nn.conv2d(X,
                         W,
                         strides=[1,1,1,1],
                         padding='SAME',
                          name="conv")
    h=tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
    
    pooled=tf.nn.max_pool(h,
                             ksize=[1, 1, embedding_size, 1],
                              strides=[1,1,1,1],
                             padding='VALID',
                             name="pooling")
    #pooled_flat=tf.contrib.layers.flatten(pooled)

with tf.name_scope("dropout"):
    h_drop=tf.nn.dropout(pooled, 0.5)
    h_drop_flattened=tf.contrib.layers.flatten(h_drop)
    
with tf.name_scope("output"):
    W = tf.Variable(tf.truncated_normal([35*n_filters, 2], stddev=0.1), name="W")
    b = tf.Variable(tf.constant(0.1, shape=[2]), name="b")
    scores = tf.nn.xw_plus_b(h_drop_flattened, W, b, name="scores")
    predictions = tf.argmax(scores, 1, name="predictions")
    
# Calculate mean cross-entropy loss
with tf.name_scope("loss"):
    #losses = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=scores)
    xentropy=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=scores)
    loss = tf.reduce_mean(xentropy)
        
with tf.name_scope("optimizer"):
    optimizer=tf.train.AdamOptimizer(0.01)
    training_op=optimizer.minimize(loss)

# Calculate Accuracy
with tf.name_scope("accuracy"):
    correct_predictions = tf.equal(predictions, y)
    accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")
    
n_epochs=500
batch_size=5
best_validation_loss=np.infty
max_checks_no_progress=20
check_without_process=0
n_batch=len(revs)//batch_size
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    start=time.time()
    for epoch in range(n_epochs):
        for batch in range(n_batch):
            X_batch, y_batch=embedding_batch(revs, W=word2vec, max_sentence_len=35, \
                                        batch_size=batch_size, batch_index=batch, embedding_size=embedding_size)
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
        training_acu=accuracy.eval(feed_dict={X:X_train_num, y:y_train})
        validation_acu=accuracy.eval(feed_dict={X:X_valid_num, y:y_valid})
        print("Epoch:", epoch, "training:", training_acu, "validation:", validation_acu)
        validation_loss=loss.eval(feed_dict={X:X_valid_num, y:y_valid})
        if(validation_loss<best_validation_loss):
            #save_path = saver.save(sess, "./my_mnist_model_0_to_4.ckpt")
            best_validation_loss=validation_loss
            check_without_process=0
        else:
            check_without_process+=1
            if check_without_process>=max_checks_no_progress:
                print("Early stopping!")
                break;
print("Training completed in ", time.time()-start)